In [1]:
'''
Reference:
https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/
'''

'\nReference:\nhttps://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/\n'

In [2]:
#for computer room 
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     \\FILE99CS\home$\G1710171\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     \\FILE99CS\home$\G1710171\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Custom function to process of store:
import json #in 2, this step is indispensable
def process_or_store(tweet):
    print(json.dumps(tweet)) #turn the tweet into a JSON type

# The Anatomy of a Tweet

In [6]:
#單個Tweet裡所包含的所有資訊
import json
with open('Cry.json', 'r') as f:
    line = f.readline()
    tweet = json.loads(line)
    #print(json.dumps(tweet, indent=4))
    
#tweet usually in unicode -> transform
enc = tweet['text'].encode('unicode-escape')

import nltk
from nltk import word_tokenize
print(word_tokenize(enc))

#\n 怎麼辦?

['RT', '@', 'choi_bts2', ':', 'I', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'Concept', 'photo', 'tonight..', '\\U0001f926\\U0001f3fb\\u200d\\u2640\\ufe0f\\n\\nI', 'think', 'I\\u2019ll', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'Concept', 'photo', 'wi\\u2026']


In [7]:
#the tweet be divided into too seperate
import re

emoticons_str = r'''
    (?:
        [:=;] #eye
        [oO\-] #nose
        [D\)\]\)\]/\\OpP] #mouth
    )'''

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

#re.compile(pattern) comile the pattern of re.
#ref: http://zwindr.blogspot.jp/2016/01/python-regular-expression.html
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.X |re.I)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.X | re.I)
#I -> ignorance: take the lowercase and uppercase as same
#X -> Verbose: make a re. more readable, ignore the '#' explantion


def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s.encode('unicode-escape'))
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower for token in tokens]
    return tokens

    
#re.search:Scan through string looking for the first 
#      location where the regular expression pattern 
#      produces a match

pre_enc = preprocess(enc)
print(pre_enc) 

['RT', '@choi_bts2', ':', 'I', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'Concept', 'photo', 'tonight', '.', '.', '\\', '\\', 'U0001f926', '\\', '\\', 'U0001f3fb', '\\', '\\', 'u200d', '\\', '\\', 'u2640', '\\', '\\', 'ufe', '0', 'f', '\\', '\\', 'n', '\\', '\\', 'nI', 'think', 'I', '\\', '\\', 'u2019ll', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'Concept', 'photo', 'wi', '\\', '\\', 'u2026']


#### 目前已完整分解字詞，並可獨立出emoji，接下來做成dataframe

In [8]:
word_list = []
with open('Cry.json') as f:
    for line in f:
        tweet = json.loads(line)
        text = tweet['text'].encode('unicode-escape')
        words = word_tokenize(text) #只留下重要字詞，需要重寫函數
        word_list.append(words) #一個超長list

from itertools import chain
word_list = list(chain.from_iterable(word_list))
word_list = [word.lower() for word in word_list] #斷完詞後的"所有"詞跟符號

### Build list of stopwords
### Take the unique word and emoji from tweet 

In [21]:
from nltk.corpus import stopwords
import string

punctuation = [string.punctuation]
stop_uni = stopwords.words('english') + punctuation + ['rt','via']

stop = [word.encode('ascii') for word in stop_uni]    
word_stop = [word for word in word_list 
            if word not in stop and
                    not word.startswith(
    ('u0','http','@','#','\\','/','*','-','{','}','[',']','+','x'
     'u2','ua','ub','uc','ud')) and
                    not word.isdigit()]
emoji = [emoji for emoji in word_list 
             if emoji.startswith(('u000'))]


In [22]:
print(word_list)

['rt', '@', 'choi_bts2', ':', 'i', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'concept', 'photo', 'tonight..', '\\u0001f926\\u0001f3fb\\u200d\\u2640\\ufe0f\\n\\ni', 'think', 'i\\u2019ll', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'concept', 'photo', 'wi\\u2026', 'rt', '@', 'faiqyuhaizad', ':', 'for', 'a', 'better', 'malaysia', ',', 'jangan', 'mengharapkan', 'perubahan', 'daripada', 'kerajaan', 'semata2', ',', 'rakyat', 'kena', 'berubah', 'gak', '.', 'sampah', 'jangan', 'bu\\u2026', 'rt', '@', 'hafizkrs', ':', 'im', 'gon', 'na', 'miss', 'this', 'guy', '\\u0001f622', 'https', ':', '//t.co/qrkxzoxhwq', 'rt', '@', 'iam_khunguy', ':', '*\\u0e1d\\u0e32\\u0e01\\u0e41\\u0e17\\u0e49\\u0e01\\u0e2b\\u0e19\\u0e48\\u0e2d\\u0e22\\u0e04\\u0e23\\u0e31\\u0e1a*\\n\\u0e23\\u0e38\\u0e48\\u0e19\\u0e1e\\u0e35\\u0e48\\u0e1c\\u0e21\\u0e40\\u0e2d\\u0e07\\n\\u0e1d\\u0e32\\u0e01\\u0e0a\\u0e48\\u0e27\\u0e22\\u0e40\\u0e2b\\u0e25\\u0e37\\u0e2d\\u0e19\\u0e49\\u0e2d\\u0e07\\u0e14\\u0e49\\u0e27\\u0e

In [11]:
#build the dataframe
word_single = list(set(word_stop))
emoji_single = list(set(emoji))

emoji_single[6] = 'u001f631'
word_single.append('somebody')
word_single = sorted(word_single)

import pandas as pd
import numpy as np

#pd.DataFrame(0,
#             columns = emoji_single,
#             index = word_single)

IndexError: list assignment index out of range

In [25]:
import json
with open('Cry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        print(json.dumps(tweet, indent=4))
   

{
    "quote_count": 0, 
    "contributors": null, 
    "truncated": false, 
    "text": "RT @choi_bts2: I feel like there will be one more Concept photo tonight.. \ud83e\udd26\ud83c\udffb\u200d\u2640\ufe0f\n\nI think I\u2019ll be late from work but if the Concept photo wi\u2026", 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "reply_count": 0, 
    "id": 994558244876992512, 
    "favorite_count": 0, 
    "entities": {
        "user_mentions": [
            {
                "id": 953663868659798017, 
                "indices": [
                    3, 
                    13
                ], 
                "id_str": "953663868659798017", 
                "screen_name": "choi_bts2", 
                "name": "Soo Choi"
            }
        ], 
        "symbols": [], 
        "hashtags": [], 
        "urls": []
    }, 
    "retweeted": false, 
    "coordinates": null, 
    "timestamp_ms": "1525956138234", 
    "source": "<a href=\"http://twitter.com/download

In [82]:
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
import pandas as pd

def text_process(text):
    
    enc = text.encode('unicode-escape')
    pro_text = preprocess(enc)

    #[emoji for emoji in pro_text if emoji.startswith('U000')]
    stop_tuple = ('U000','RT','@',':','/','\\''','.','u2','http','u0',
                 'ua','ub','uc','ud')
    emoji_list = [emoji for emoji in pro_text if emoji.startswith('U00')]
    pro_text = [word for word in pro_text if not word.startswith(stop_tuple)
                            and not word.isdigit()
                            and not len(word) < 2]
    return emoji_list, FreqDist(pro_text)

def merge_dataframe(emoji_list, pro_text, current_df):
    fdist = FreqDist(pro_text)
    
    fdist_list = []
    for i in range(len(emoji_list)):
        fdist_list.append(fdist)
    
    new_df = pd.DataFrame(fdist_list, index = emoji_list)
    df = pd.concat([current_df, new_df]).T
    return df 

a,b = text_process(text)
test_df = pd.DataFrame([1,2,3])
merge_dataframe(a,b,test_df)

,0,1,2,U0001f622
0,1.0,2.0,3.0,NaN
Negeri,NaN,NaN,NaN,1.0
Sembilan,NaN,NaN,NaN,1.0
anak-anak,NaN,NaN,NaN,1.0
berjaya,NaN,NaN,NaN,1.0
nAllahuakbar,NaN,NaN,NaN,1.0
nak,NaN,NaN,NaN,1.0
saya,NaN,NaN,NaN,1.0


In [83]:
import json
import numpy as np
cry_list = []

with open('Cry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        emoji_list, pro_text = text_process(tweet['text'])
        cry_list.append([emoji_list, pro_text])
        #df = merge_dataframe(emoji_list, pro_text, df_emoji)

cry_list

[[['U0001f926', 'U0001f3fb'],
  FreqDist({'Concept': 2,
            'be': 2,
            'but': 1,
            'feel': 1,
            'from': 1,
            'if': 1,
            'late': 1,
            'like': 1,
            'more': 1,
            'nI': 1,
            'one': 1,
            'photo': 2,
            'the': 1,
            'there': 1,
            'think': 1,
            'tonight': 1,
            'ufe': 1,
            'wi': 1,
            'will': 1,
            'work': 1})],
 [[],
  FreqDist({'2,': 1,
            'For': 1,
            'Malaysia': 1,
            'Sampah': 1,
            'berubah': 1,
            'better': 1,
            'bu': 1,
            'daripada': 1,
            'gak': 1,
            'jangan': 2,
            'kena': 1,
            'kerajaan': 1,
            'mengharapkan': 1,
            'perubahan': 1,
            'rakyat': 1,
            'semata': 1})],
 [['U0001f622'],
  FreqDist({'gonna': 1, 'guy': 1, 'im': 1, 'miss': 1, 'this': 1})],
 [['U0001f622'],

In [76]:
angry_list = []

with open('Angry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        emoji_list, pro_text = text_process(tweet['text'])
        angry_list.append([emoji_list, pro_text])
        
angry_list

[[['U0001f621'],
  FreqDist({'Instagrami': 1, 'be': 1, 'da': 1, 'xe7': 1, 'xf6kerttiniz': 1})],
 [['U0001f621'],
  FreqDist({'#instagramdown': 1,
            'Avevo': 1,
            'bello': 1,
            'fatto': 1,
            'molto': 1,
            'post': 1,
            'un': 1})],
 [['U0001f621'],
  FreqDist({'#Elections': 1,
            '#SarkaraBadalisiBJPGelli': 1,
            'Trying': 1,
            'cant': 1,
            'create': 1,
            'fake': 1,
            'get': 1,
            'real': 1,
            'style': 1,
            'to': 1,
            'voters': 1,
            'votes': 2,
            'when': 1,
            'win': 1})],
 [[],
  FreqDist({'Privileged': 1,
            'Slap': 1,
            'Yeah': 1,
            'and': 1,
            'arrest': 1,
            'bombing': 1,
            'capitol': 1,
            'clas': 1,
            'kaines': 1,
            'kid': 1,
            'of': 1,
            'on': 1,
            'resisting': 1,
            'smoke'

In [57]:
df = pd.DataFrame(np.zeros(1))
pd.concat([pd.DataFrame(np.zeros(10)), 
          pd.DataFrame(np.ones(5))])

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [ ]:
#每一行都造一個dataframe然後再合併

import json
with open('Angry.json', 'r') as f:
    line = f.readline()
    tweet = json.loads(line)
    #print(json.dumps(tweet, indent=4))
    
#tweet usually in unicode -> transform
enc = tweet['text'].encode('unicode-escape')

pro_text = preprocess(enc)

#[emoji for emoji in pro_text if emoji.startswith('U000')]
stop_tuple = ('U000','RT','@',':','/','\\''','.','u2')
emoji_list = [emoji for emoji in pro_text if emoji.startswith('U00')]
pro_text = [word for word in pro_text if not word.startswith(stop_tuple)
                            and not word.isdigit()
                            and not len(word) < 2]

fdist1 = FreqDist(pro_text)
fdist1

test_df2 = pd.DataFrame(fdist1,index = emoji_list).T
test_df2

In [ ]:
pd.concat([test_df1, test_df2])


In [41]:
test_df1

NameError: name 'test_df1' is not defined